In [15]:
import pandas as pd 
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from xgboost import XGBRFRegressor
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense
from prettytable import PrettyTable

In [2]:
missing_df = pd.read_csv('missingmarks_clean_data.csv')

In [3]:
missing_df.head()

,institution,system,data_loss_gen,data_loss_human,data_loss_machine,data_loss_hybrid,system_error_chance,feature
0,10602,1,2,3.0,4.0,1.0,1.0,0
1,10612,1,2,3.0,3.0,1.0,1.0,0
2,10597,1,2,1.0,1.0,2.0,1.0,1
3,10597,2,2,2.0,4.0,2.0,1.0,0
4,10597,1,1,1.0,2.0,1.0,1.0,1


In [21]:
missing_df.isnull().values.any()

False

In [22]:
missing_df.isnull().sum().sum()

0

In [20]:
missing_df.dtypes

institution              int64
system                   int64
data_loss_gen            int64
data_loss_human        float64
data_loss_machine      float64
data_loss_hybrid       float64
system_error_chance    float64
feature                  int64
dtype: object

In [5]:
train_cols = ['data_loss_gen', 'data_loss_human', 'data_loss_machine', 'data_loss_hybrid', 'system_error_chance']
x = missing_df[train_cols]
y = missing_df['feature']

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=25)

In [7]:
scaler = StandardScaler()

X_train_scale = scaler.fit_transform(x_train)
X_test_scale = scaler.fit_transform(x_test)

In [12]:
from sklearn.svm import SVC
import numpy as np

In [14]:
def auto_train (x,y, model_alg):
    if model_alg == 'lr':
        model = LinearRegression()
    elif model_alg == 'sv':
        model = SVC()
    elif model_alg == 'xgb':
        model = XGBRFRegressor()
    elif model_alg == 'rfr':
        model = RandomForestRegressor()
    elif model_alg == 'rfc':
        model = RandomForestClassifier()
        
    model.fit(X_train_scale, np.log(y))
    
    return model

In [16]:
def auto_evaluate(model, x, y):
    predictions = model.predict(x)
    predictions = np.exp(predictions)
    
    mse = mean_squared_error(y, predictions)
    mae = mean_absolute_error(y, predictions)
    mape = mean_absolute_percentage_error(y, predictions)
    
    mse = round(mse, 2)
    mae = round(mae, 2)
    mape = round(mape, 2)
    
    return [mse, mae, mape]

In [17]:
linear = auto_train(X_train_scale, y_train, 'lr')
svm = auto_train(X_train_scale, y_train, 'sv')
xgboost = auto_train(X_train_scale, y_train, 'xgb')
random_forest_reg = auto_train(X_train_scale, y_train, 'rfr')
random_forest_cls = auto_train(X_train_scale, y_train, 'rfc')

C:\Users\Wessware\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pandas\core\arraylike.py:358: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').